In [1]:
import os
import re
import glob
import pickle
import numpy as np
import pandas as pd
import pandas.io.common
import warnings
warnings.filterwarnings('ignore')

In [42]:
#### File Path and Variables
loc_path = r'/Users/ritik.pareek/Desktop/' 
path = loc_path + r'/Causal Model/01. Raw Data/04. Network Health Data/'
pickle_path = loc_path + 'Causal Model/02. Processed Data/04. Network Health Data/'
### Input for the Circle 
Circle= 'bihar'

monthname = "Feb"
year = "2023"

In [7]:
#### Collating Files 
# currently we have colated file but in future run we have to colate the files


allFiles = glob.glob(path + "*/*.parquet")
lst=[]
df_master=pd.DataFrame()
for i in allFiles:
    df=pd.read_parquet(i,engine='pyarrow')
    #print(df.shape)
    df_master=df_master.append(df)
    #print(df_master.shape)

In [8]:
df_master.shape

(995120, 6)

In [9]:
df_master.columns

Index(['partition_date', 'city', 'component', 'hour', 'avg_bad_hsi_pct',
       'avg_bad_voice_pct'],
      dtype='object')

# Removing rows where hour is null

In [10]:
df_master['hour'].unique()

array([ 5, 19, 18, 22, 21, 23, 13,  3,  9, 11,  0, 16, 14, 15,  7,  4, 20,
       17, 12,  8,  2, 10,  1,  6], dtype=int32)

In [11]:
# Null count
df_master[df_master['hour'].isnull()].shape[0]

0

In [18]:
# dropping the null values
df_master.dropna(subset=['hour','city','component'], how='any', inplace=True)

In [20]:
# index reseting
df_master.reset_index(inplace=True, drop=True)

In [21]:
df_master.shape

(994952, 6)

In [22]:
## Null count
df_master.isnull().sum()

partition_date           0
city                     0
component                0
hour                     0
avg_bad_hsi_pct      95693
avg_bad_voice_pct      689
dtype: int64

In [23]:
df_master.head()

,partition_date,city,component,hour,avg_bad_hsi_pct,avg_bad_voice_pct
0,2022-11-03,motihari,151.0,5,8.054286,22.754500
1,2022-11-03,sasaram,1.0,5,16.767143,15.845909
2,2022-11-03,samastipur,72.0,19,24.924286,10.074286
3,2022-11-03,muzaffarpur,54.0,18,7.685000,20.361809
4,2022-11-03,ranchi,122.0,22,14.585333,20.151000


# Defining the data types

In [25]:
df_master['circle']="bihar"

In [26]:
df_master.dtypes

partition_date        object
city                  object
component            float64
hour                   int32
avg_bad_hsi_pct      float64
avg_bad_voice_pct    float64
circle                object
dtype: object

In [27]:
df_master["partition_date"] =  pd.to_datetime(df_master["partition_date"], format="%Y-%m-%d")
df_master['circle'] = df_master['circle'].astype('object')
df_master['city'] = df_master['city'].astype('object')
df_master['component'] = df_master['component'].astype('Int64')
df_master['hour'] = df_master['hour'].astype('Int64')
df_master['avg_bad_hsi_pct'] = df_master['avg_bad_hsi_pct'].astype('float64')
df_master['avg_bad_voice_pct'] = df_master['avg_bad_voice_pct'].astype('float64')

In [28]:
df_master.head()

,partition_date,city,component,hour,avg_bad_hsi_pct,avg_bad_voice_pct,circle
0,2022-11-03,motihari,151,5,8.054286,22.754500,bihar
1,2022-11-03,sasaram,1,5,16.767143,15.845909,bihar
2,2022-11-03,samastipur,72,19,24.924286,10.074286,bihar
3,2022-11-03,muzaffarpur,54,18,7.685000,20.361809,bihar
4,2022-11-03,ranchi,122,22,14.585333,20.151000,bihar


# Rounding off the data

In [29]:
df_master=df_master.round(2)

In [30]:
df_master.head()

,partition_date,city,component,hour,avg_bad_hsi_pct,avg_bad_voice_pct,circle
0,2022-11-03,motihari,151,5,8.05,22.75,bihar
1,2022-11-03,sasaram,1,5,16.77,15.85,bihar
2,2022-11-03,samastipur,72,19,24.92,10.07,bihar
3,2022-11-03,muzaffarpur,54,18,7.68,20.36,bihar
4,2022-11-03,ranchi,122,22,14.59,20.15,bihar


# Setting the column seq

In [31]:
print(df_master.shape)
print(df_master.columns)

(994952, 7)
Index(['partition_date', 'city', 'component', 'hour', 'avg_bad_hsi_pct',
       'avg_bad_voice_pct', 'circle'],
      dtype='object')


In [32]:
df_master=df_master[['partition_date','circle','city', 'component', 'hour', 'avg_bad_hsi_pct', 'avg_bad_voice_pct']]

In [33]:
df_master.head()

,partition_date,circle,city,component,hour,avg_bad_hsi_pct,avg_bad_voice_pct
0,2022-11-03,bihar,motihari,151,5,8.05,22.75
1,2022-11-03,bihar,sasaram,1,5,16.77,15.85
2,2022-11-03,bihar,samastipur,72,19,24.92,10.07
3,2022-11-03,bihar,muzaffarpur,54,18,7.68,20.36
4,2022-11-03,bihar,ranchi,122,22,14.59,20.15


# Removing right Spaces from the columns

In [34]:
print('Removing right Spaces from the columns')
df_master.columns = df_master.columns.str.rstrip()
df_master.columns = df_master.columns.str.lstrip()
df_master.columns = df_master.columns.str.capitalize()

Removing right Spaces from the columns


# Remove white space and Drop duplicates

In [35]:
## Pre Processing
def remove_whitespaces(df):
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    df.columns = df.columns.str.strip()
    return df

def drop_duplicate_records(df):
    df.drop_duplicates()
    return df

In [36]:
df_master=remove_whitespaces(df_master)
df_master=drop_duplicate_records(df_master)

In [37]:
df_master.head()

,Partition_date,Circle,City,Component,Hour,Avg_bad_hsi_pct,Avg_bad_voice_pct
0,2022-11-03,bihar,motihari,151,5,8.05,22.75
1,2022-11-03,bihar,sasaram,1,5,16.77,15.85
2,2022-11-03,bihar,samastipur,72,19,24.92,10.07
3,2022-11-03,bihar,muzaffarpur,54,18,7.68,20.36
4,2022-11-03,bihar,ranchi,122,22,14.59,20.15


In [38]:
# no of unique cities
df_master['City'].unique()

array(['motihari', 'sasaram', 'samastipur', 'muzaffarpur', 'ranchi',
       'bokaro_steel_city', 'ramgarh', 'gaya', 'patna', 'bihar_sharif',
       'sahibganj', 'darbhanga', 'madhubani', 'hajipur', 'khagaria',
       'bhagalpur', 'araria', 'daltonganj', 'jamui', 'arrah', 'katihar',
       'dhanbad', 'kishanganj', 'jamshedpur', 'supaul', 'chaibasa',
       'buxar', 'nawada', 'begusarai', 'saunda', 'jehanabad',
       'lakhisarai', 'deoghar', 'mokameh', 'hazaribag', 'saharsa',
       'aurangabad_bi', 'chirkunda', 'giridih', 'chapra', 'siwan',
       'bagaha', 'munger', 'bettiah', 'jamalpur', 'dehri', 'dumka',
       'sitamarhi', 'phusro', 'gopalganj', 'rajgir', 'jhumri_tilaiya',
       'chakradharpur', 'barauni'], dtype=object)

In [39]:
# no of comp in each city
df_master[['City','Component']].groupby('City').nunique()

,Component
City,
araria,80
arrah,123
aurangabad_bi,70
bagaha,47
barauni,82
begusarai,142
bettiah,110
bhagalpur,234
bihar_sharif,130


In [40]:
## Null count
df_master.isnull().sum()

Partition_date           0
Circle                   0
City                     0
Component                0
Hour                     0
Avg_bad_hsi_pct      95693
Avg_bad_voice_pct      689
dtype: int64

In [41]:
df_master[['Avg_bad_hsi_pct','Avg_bad_voice_pct']].describe()

,Avg_bad_hsi_pct,Avg_bad_voice_pct
count,899259.000000,994263.000000
mean,12.017138,18.358149
std,8.706622,7.661196
min,0.000000,0.000000
25%,6.070000,13.320000
50%,9.910000,17.180000
75%,15.710000,22.050000
max,100.000000,100.000000


In [45]:
### Saving in Pre Processing Folder
df_master.to_pickle(pickle_path+'{}{}_Network_Health_data.csv'.format(monthname,year))
print('Data succesfully stored in:',pickle_path+'{}{}_Network_Health_data.csv'.format(monthname,year))

Data succesfully stored in: /Users/ritik.pareek/Desktop/Causal Model/02. Processed Data/04. Network Health Data/Feb2023_Network_Health_data.csv
